# Hoeken berekenen

## Functies

In [2]:
import numpy as np
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from dateutil import parser, relativedelta
from IPython.display import display

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 1000)

# hip to origin, recalculate all other points with that translation
def x_translation (x_ver, row):
    x_ch = row['x']
    x_n = x_ch - x_ver
    return  x_n

def z_translation (z_ver, row):
    z_ch = row['z']
    z_n = z_ch - z_ver
    return z_n

# calculate arc that the body needs to rotate to get both hips on the same z-coordinate
def define_arc(frame):
    x_right = float(frame.loc[frame['jointName'] == "HipRight", 'x_schuif'])
    x_left = float(frame.loc[frame['jointName'] == "HipLeft", 'x_schuif'])
    z_right = float(frame.loc[frame['jointName'] == "HipRight", 'z_schuif'])
    z_left = float(frame.loc[frame['jointName'] == "HipLeft", 'z_schuif'])
    
    arc = math.atan(abs(z_right-z_left)/abs(x_right-x_left))
    return arc

# rotate a given point and calculate new x and new z
def rotate_body_x(row, arc):
    new_x = (math.cos(arc) * float(row['x_schuif'])) + (math.sin(arc) * float(row['z_schuif']))
    return new_x

def rotate_body_z(row, arc):
    new_z = (math.cos(arc) * float(row['z_schuif'])) - (math.sin(arc) * float(row['x_schuif'])) 
    return new_z

################
# Calculate arc
################

# exercise 1
def arc_elbow(frame, side):
    spineShoulder = np.matrix([[float(frame.loc[frame['jointName'] == 'SpineShoulder', 'x'])], 
                               [float(frame.loc[frame['jointName'] == 'SpineShoulder', 'y'])]])
    spineMid = np.matrix([[float(frame.loc[frame['jointName'] == 'SpineMid', 'x'])], 
                               [float(frame.loc[frame['jointName'] == 'SpineMid', 'y'])]])
    
    spineMid_new = spineMid - spineShoulder
    
    if side == 'r':
        # right side
        jointName_shoulder = "ShoulderRight"
        jointName_elbow = "ElbowRight"
    else:
        # left side
        jointName_shoulder = "ShoulderLeft"
        jointName_elbow = "ElbowLeft"
    
    shoulder = np.matrix([[float(frame.loc[frame['jointName'] == jointName_shoulder, 'x'])], 
                        [float(frame.loc[frame['jointName'] == jointName_shoulder, 'y'])]])
    elbow = np.matrix([[float(frame.loc[frame['jointName'] == jointName_elbow, 'x'])], 
                        [float(frame.loc[frame['jointName'] == jointName_elbow, 'y'])]])

    elbow_new = elbow - shoulder

    sum_vectors = np.dot(np.transpose(spineMid_new), elbow_new)
    multiplication_lengths = np.linalg.norm(spineMid_new) * np.linalg.norm(elbow_new)

    cos_alpha = sum_vectors / multiplication_lengths
    alpha = np.arccos(cos_alpha) * 180 / math.pi
    return alpha

# exersice 2
def arc_elbow_front(frame, side):
    spineShoulder = np.matrix([[float(frame.loc[frame['jointName'] == 'SpineShoulder', 'z'])], 
                               [float(frame.loc[frame['jointName'] == 'SpineShoulder', 'y'])]])
    spineMid = np.matrix([[float(frame.loc[frame['jointName'] == 'SpineMid', 'z'])], 
                               [float(frame.loc[frame['jointName'] == 'SpineMid', 'y'])]])
    
    spineMid_new = spineMid - spineShoulder
    
    if side == 'r':
        # right side
        jointName_shoulder = "ShoulderRight"
        jointName_elbow = "ElbowRight"
    else:
        # left side
        jointName_shoulder = "ShoulderLeft"
        jointName_elbow = "ElbowLeft"
    
    shoulder = np.matrix([[float(frame.loc[frame['jointName'] == jointName_shoulder, 'z'])], 
                        [float(frame.loc[frame['jointName'] == jointName_shoulder, 'y'])]])
    elbow = np.matrix([[float(frame.loc[frame['jointName'] == jointName_elbow, 'z'])], 
                        [float(frame.loc[frame['jointName'] == jointName_elbow, 'y'])]])

    elbow_new = elbow - shoulder

    sum_vectors = np.dot(np.transpose(spineMid_new), elbow_new)
    multiplication_lengths = np.linalg.norm(spineMid_new) * np.linalg.norm(elbow_new)

    cos_alpha = sum_vectors / multiplication_lengths
    alpha = np.arccos(cos_alpha) * 180 / math.pi
    return alpha

# exercise 3
def arc_elbow_above(frame, side):    
    if side == 'r':
        # right side
        jointName_elbow = "ShoulderRight"
        jointName_wrist = "WristRight"
    else:
        # left side
        jointName_elbow = "ShoulderLeft"
        jointName_wrist = "WristLeft"
    
    elbow = np.matrix([[float(frame.loc[frame['jointName'] == jointName_elbow, 'x'])], 
                        [float(frame.loc[frame['jointName'] == jointName_elbow, 'z'])]])
    wrist = np.matrix([[float(frame.loc[frame['jointName'] == jointName_wrist, 'x'])], 
                        [float(frame.loc[frame['jointName'] == jointName_wrist, 'z'])]])
    
    wrist_new = wrist - elbow
    front_vector = np.matrix([[0],[-1]])
    
    sum_vectors = np.dot(np.transpose(front_vector), wrist_new)
    multiplication_lengths = np.linalg.norm(front_vector) * np.linalg.norm(wrist_new)

    cos_alpha = sum_vectors / multiplication_lengths
    alpha = np.arccos(cos_alpha) * 180 / math.pi
    return alpha

def relative_time(row, frame):
    index = frame.index.get_loc(row.name)
    
    if index == 0:
        return 0
    
    first_row = frame.iloc[0]
    time1 = row['time']
    time2 = first_row['time']
    
    dt1 = parser.parse(time1)
    dt2 = parser.parse(time2)
    delta = relativedelta.relativedelta(dt1, dt2)
    microseconds = delta.seconds * 1000000 + delta.microseconds
    return microseconds

def normalized_time(row, frame):
    total_time = frame['relativeTime'].max()
    
    relative_time = row['relativeTime']
    normalized = relative_time/total_time
    return normalized

height has been deprecated.



## Exercise 1

In [3]:
# import math 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import trapz

# import data 
# HIER INVULLEN WAAR JOUW DATA STAAT
df = pd.read_csv('cleanedData_ex1.csv')      #('combined_P1-5_E1.csv')    # ('/data/pepper/combined.csv')

for person in range(1, 6):
    print(str(person))
    # gets data from 1 person
    one_p = df.loc[df['pNum'] == person]
    one_e = one_p

    ###################
    # ROTATE BODY
    ###################
    # create empty DataFrame
    index = np.arange(0)
    frames = pd.DataFrame(index=index)
    frames = frames.fillna(0) # with 0s rather than NaNs

    # get min and max frameNum
    frame_max = int(one_e['frameNum'].max())
    frame_min = int(one_e['frameNum'].min())

    found = False
    
    for i in range(frame_min, frame_max + 1):   
        frame = one_e.loc[one_e['frameNum'] == i]

        try:
            x_ver = float(frame.loc[frame['jointName'] == "HipRight", 'x'])
            z_ver = float(frame.loc[frame['jointName'] == "HipRight", 'z'])
            
#             if float(frame.loc[frame['jointName'] == "HipRight", 'z']) < float(frame.loc[frame['jointName'] == "HipLeft", 'z']):
#                 # rotate clockwise
#                 clock = -1
#             else:
#                 # rotate - clockwise
#                 clock = 1

            # schuiven
            frame['x_schuif'] = frame.apply (lambda row: x_translation(x_ver, row),axis=1)
            frame['z_schuif'] = frame.apply (lambda row: z_translation(z_ver, row),axis=1)

            #draaien
#             arc = define_arc(frame)
#             frame['x'] = frame.apply (lambda row: rotate_body_x(row, arc * clock),axis=1)
#             frame['z'] = frame.apply (lambda row: rotate_body_z(row, arc * clock),axis=1)

            #samenvoegen
            # all frames from one person are rotated
            frames = frames.append(frame)
            found = True
        except:
            found = False
            print('BUG')
        
    ###################
    # CALCULATE ARCS
    ###################
    if found == True:
        # create empty DataFrame
        index = np.arange(0)
        columns = ['pNum', 'eNum', 'frameNum', 'time', 'arc_right', 'arc_left']
        arcs = pd.DataFrame(index=index, columns=columns)
        arcs = arcs.fillna(0) # with 0s rather than NaNs

        frame_max_2 = frames['frameNum'].max()
        frame_min_2 = frames['frameNum'].min()

        for i in range(frame_min_2, frame_max_2 + 1): 
            frame = frames.loc[frames['frameNum'] == i]
            right_arc = arc_elbow(frame, 'r')
            left_arc = arc_elbow(frame, 'l')
            time = frame.loc[frame['jointName'] == 'SpineBase', 'time']
            time = time[time.first_valid_index()]
            row = {'pNum': person, 'eNum': 1, 'frameNum': i, 'time': time, 'arc_right': right_arc[0,0], 'arc_left': left_arc[0,0]}
            arcs = arcs.append(row, ignore_index=True)

        ##################
        # NORMALIZE TIME
        ##################

        arcs['relativeTime'] = arcs.apply (lambda row: relative_time(row, arcs),axis=1)
        arcs['normalized'] = arcs.apply (lambda row: normalized_time(row, arcs), axis=1)
    
        # area under plot
        area_right = trapz(arcs['arc_right'], arcs['normalized'])
        area_left = trapz(arcs['arc_left'], arcs['normalized'])
        
        display(arcs)
        
#         plt.plot(arcs['normalized'], arcs['arc_left'])
#         plt.plot(arcs['normalized'], arcs['arc_right'])
        
#         plt.xlabel('tijd (relatief)')
#         plt.ylabel('graden')
#         plt.ylim((0,180))
#         plt.legend()
#         # HIER INVULLEN WAAR JE GRAFIEKEN OPGESLAGEN MOETEN WORDEN
#         plt.savefig('/home/13040367/notebooks/Plots/Exercise_1/grafiek_P' + str(person) + '_ex1')    
#         plt.show()
    else:
        print('BUG')

1


/opt/jupyterhub/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/jupyterhub/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,pNum,eNum,frameNum,time,arc_right,arc_left,relativeTime,normalized
0,1,1,668,20:52:59.5967098,15.353660,10.135973,0,0.000000
1,1,1,669,20:52:59.6297118,15.361560,10.027907,33002,0.011123
2,1,1,670,20:52:59.6967252,15.317326,10.137568,100016,0.033709
3,1,1,671,20:52:59.7297256,15.136617,10.223996,133016,0.044831
4,1,1,672,20:52:59.7967259,14.904537,10.377943,200016,0.067412
5,1,1,673,20:52:59.8297180,14.741971,10.486339,233009,0.078532
6,1,1,674,20:52:59.8967290,14.222856,11.226056,300020,0.101117
7,1,1,675,20:52:59.9297121,14.519971,11.526580,333003,0.112233
8,1,1,676,20:52:59.9638000,15.265588,12.664344,367091,0.123722
9,1,1,677,20:53:00.0297239,19.512000,22.972408,433014,0.145941


2


,pNum,eNum,frameNum,time,arc_right,arc_left,relativeTime,normalized
0,2,1,647,20:56:13.9628621,10.251619,9.981129,0,0.000000
1,2,1,648,20:56:13.9957780,9.973468,9.709516,32916,0.011485
2,2,1,649,20:56:14.0287773,9.885516,9.625346,65915,0.023000
3,2,1,650,20:56:14.0628557,9.905017,9.686454,99993,0.034891
4,2,1,651,20:56:14.0957559,10.668959,8.985340,132893,0.046371
5,2,1,652,20:56:14.1287756,11.302981,9.093561,165913,0.057892
6,2,1,653,20:56:14.1628498,13.667787,10.355684,199987,0.069782
7,2,1,654,20:56:14.1957610,17.158902,13.335918,232899,0.081266
8,2,1,655,20:56:14.2287756,21.678638,19.296555,265913,0.092786
9,2,1,656,20:56:14.2628505,27.383961,24.203806,299988,0.104676


...

,pNum,eNum,frameNum,time,arc_right,arc_left,relativeTime,normalized
0,3,1,466,20:58:54.8279784,9.627231,13.426380,0,0.000000
1,3,1,467,20:58:54.8621776,9.687265,13.353279,34199,0.007656
2,3,1,468,20:58:54.8949712,9.637915,13.292266,66993,0.014997
3,3,1,469,20:58:54.9279704,9.610434,13.331080,99992,0.022385
4,3,1,470,20:58:54.9620955,9.620247,13.310307,134117,0.030024
5,3,1,471,20:58:54.9949889,9.735290,13.232598,167010,0.037388
6,3,1,472,20:58:55.0279885,9.774197,13.290393,200010,0.044775
7,3,1,473,20:58:55.0620872,9.830413,13.271211,234109,0.052409
8,3,1,474,20:58:55.0949778,9.902776,13.295193,266999,0.059772
9,3,1,475,20:58:55.1279719,10.010454,13.356754,299993,0.067158


4


,pNum,eNum,frameNum,time,arc_right,arc_left,relativeTime,normalized
0,4,1,660,21:02:25.4269443,11.847472,11.217106,0,0.000000
1,4,1,661,21:02:25.4610180,11.704570,11.212708,34074,0.004689
2,4,1,662,21:02:25.4939600,11.766719,11.112823,67016,0.009222
3,4,1,663,21:02:25.5269604,11.788035,11.095720,100016,0.013763
4,4,1,664,21:02:25.5610180,11.789217,11.083050,134074,0.018450
5,4,1,665,21:02:25.5939402,11.816286,11.071503,166996,0.022980
6,4,1,666,21:02:25.6269394,11.841897,11.064843,199995,0.027521
7,4,1,667,21:02:25.6610191,11.865242,11.083867,234075,0.032211
8,4,1,668,21:02:25.6939603,11.889291,11.130005,267016,0.036744
9,4,1,669,21:02:25.7269564,12.016923,11.353718,300012,0.041284


5


,pNum,eNum,frameNum,time,arc_right,arc_left,relativeTime,normalized
0,5,1,625,21:04:50.8933041,11.618036,16.351448,0,0.000000
1,5,1,626,21:04:50.9603246,11.601325,15.996881,67020,0.008818
2,5,1,627,21:04:50.9933250,11.625753,15.662408,100021,0.013161
3,5,1,628,21:04:51.0603234,11.468887,15.086591,167019,0.021976
4,5,1,629,21:04:51.0933234,11.436756,14.886247,200019,0.026318
5,5,1,630,21:04:51.1603226,11.284348,14.852829,267018,0.035134
6,5,1,631,21:04:51.1933226,11.320701,14.754012,300018,0.039476
7,5,1,632,21:04:51.2262472,11.326443,14.735897,332943,0.043808
8,5,1,633,21:04:51.2933233,11.364913,14.960367,400019,0.052634
9,5,1,634,21:04:51.3603217,11.423825,15.494576,467017,0.061450


## Exercise 2

In [3]:
import math 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# import data 
df = pd.read_csv('cleanedData_ex2.csv')      #('combined_P1-5_E1.csv')    # ('/data/pepper/combined.csv')

for person in range(1, 6):
    print(str(person))
    # gets data from 1 person
    one_p = df.loc[df['pNum'] == person]
    one_e = one_p # .loc[one_p['eNum'] == 2]
#     one_e = one_e.loc[one_e['frameNum'] > 195]
#     one_e = one_e.loc[one_e['frameNum'] < 250]

    ###################
    # ROTATE BODY
    ###################
    # create empty DataFrame
    index = np.arange(0)
    frames = pd.DataFrame(index=index)
    frames = frames.fillna(0) # with 0s rather than NaNs

    # get min and max frameNum
    frame_max = int(one_e['frameNum'].max())
    frame_min = int(one_e['frameNum'].min())

    found = False
    
    for i in range(frame_min, frame_max + 1):   
        frame = one_e.loc[one_e['frameNum'] == i]

        try:
            x_ver = float(frame.loc[frame['jointName'] == "HipRight", 'x'])
            z_ver = float(frame.loc[frame['jointName'] == "HipRight", 'z'])

#             if float(frame.loc[frame['jointName'] == "HipRight", 'z']) < float(frame.loc[frame['jointName'] == "HipLeft", 'z']):
#                 # rotate clockwise
#                 clock = -1
#             else:
#                 # rotate - clockwise
#                 clock = 1
            
            # schuiven
            frame['x_schuif'] = frame.apply (lambda row: x_translation(x_ver, row),axis=1)
            frame['z_schuif'] = frame.apply (lambda row: z_translation(z_ver, row),axis=1)

            #draaien
#             arc = define_arc(frame)
#             frame['x_rotated'] = frame.apply (lambda row: rotate_body_x(row, arc * clock),axis=1)
#             frame['z_rotated'] = frame.apply (lambda row: rotate_body_z(row, arc * clock),axis=1)

            #samenvoegen
            # all frames from one person are rotated
            frames = frames.append(frame)
            found = True
        except:
            found = False
            frame['x_schuif'] = frame.apply (lambda row: x_translation(x_ver, row),axis=1)
            print('BUG 1')
        
    ###################
    # CALCULATE ARCS
    ###################
    if found == True:
        # create empty DataFrame
        index = np.arange(0)
        columns = ['pNum', 'eNum', 'frameNum', 'time', 'arc_right', 'arc_left']
        arcs = pd.DataFrame(index=index, columns=columns)
        arcs = arcs.fillna(0) # with 0s rather than NaNs

        frame_max_2 = frames['frameNum'].max()
        frame_min_2 = frames['frameNum'].min()

        for i in range(frame_min_2, frame_max_2 + 1): 
            frame = frames.loc[frames['frameNum'] == i]
            right_arc = arc_elbow_front(frame, 'r')
            left_arc = arc_elbow_front(frame, 'l')
            time = frame.loc[frame['jointName'] == 'SpineBase', 'time']
            time = time[time.first_valid_index()]
            row = {'pNum': person, 'eNum': 2, 'frameNum': i, 'time': time, 'arc_right': right_arc[0,0], 'arc_left': left_arc[0,0]}
            arcs = arcs.append(row, ignore_index=True)

        ##################
        # NORMALIZE TIME
        ##################
        arcs['relativeTime'] = arcs.apply (lambda row: relative_time(row, arcs),axis=1)
        arcs['normalized'] = arcs.apply (lambda row: normalized_time(row, arcs), axis=1)
        
        display(arcs)
        
#         plt.plot(arcs['normalized'], arcs['arc_left'])
#         plt.plot(arcs['normalized'], arcs['arc_right'])
        
#         plt.xlabel('tijd (relatief)')
#         plt.ylabel('graden')
#         plt.ylim((0,180))
#         plt.legend()
#         plt.savefig('/home/13040367/notebooks/Plots/Exercise_2/grafiek_P' + str(person) + '_ex2')    # and Grafieken_3
#         plt.show()
    else:
        print('BUG 2')

1


/opt/jupyterhub/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/jupyterhub/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,pNum,eNum,frameNum,time,arc_right,arc_left,relativeTime,normalized
0,1,2,193,20:53:23.0296114,12.384806,12.867502,0,0.000000
1,1,2,194,20:53:23.0636863,11.514448,12.302250,34075,0.015725
2,1,2,195,20:53:23.0965951,10.856038,12.226982,66984,0.030911
3,1,2,196,20:53:23.1636875,11.077940,14.712247,134076,0.061872
4,1,2,197,20:53:23.1966117,17.602798,18.860428,167000,0.077066
5,1,2,198,20:53:23.2636783,33.460120,56.554991,234067,0.108015
6,1,2,199,20:53:23.2966034,45.341278,62.174313,266992,0.123209
7,1,2,200,20:53:23.3296109,58.984515,68.643421,299999,0.138441
8,1,2,201,20:53:23.3636779,69.074164,76.655171,334066,0.154162
9,1,2,202,20:53:23.3966092,80.656765,92.267320,366998,0.169359


2


,pNum,eNum,frameNum,time,arc_right,arc_left,relativeTime,normalized
0,2,2,173,20:56:38.2286386,0.750698,1.011940,0,0.000000
1,2,2,174,20:56:38.2627112,0.523657,0.864655,34073,0.005908
2,2,2,175,20:56:38.2956386,0.080580,0.369619,67000,0.011618
3,2,2,176,20:56:38.3286449,1.945029,0.752399,100006,0.017341
4,2,2,177,20:56:38.3627131,4.599568,2.237549,134075,0.023249
5,2,2,178,20:56:38.3956373,8.093211,5.524450,166999,0.028958
6,2,2,179,20:56:38.4286385,11.703685,9.092546,200000,0.034680
7,2,2,180,20:56:38.4627115,17.808264,13.676180,234073,0.040589
8,2,2,181,20:56:38.4956365,22.365993,17.746198,266998,0.046298
9,2,2,182,20:56:38.5286373,27.088193,23.513276,299999,0.052020


...

,pNum,eNum,frameNum,time,arc_right,arc_left,relativeTime,normalized
0,3,2,117,20:59:15.5948876,2.556996,2.459217,0,0.000000
1,3,2,118,20:59:15.6278884,2.130520,3.118022,33001,0.008391
2,3,2,119,20:59:15.6619626,1.359147,4.344756,67075,0.017054
3,3,2,120,20:59:15.6948876,0.240816,6.413617,100000,0.025426
4,3,2,121,20:59:15.7278742,2.236970,9.246218,132987,0.033813
5,3,2,122,20:59:15.7619641,4.628844,13.713398,167077,0.042481
6,3,2,123,20:59:15.7948879,7.954036,17.240369,200000,0.050852
7,3,2,124,20:59:15.8278836,11.459889,21.872115,232996,0.059242
8,3,2,125,20:59:15.8619609,16.488776,27.163295,267073,0.067906
9,3,2,126,20:59:15.8948725,20.251751,33.878105,299985,0.076274


4


,pNum,eNum,frameNum,time,arc_right,arc_left,relativeTime,normalized
0,4,2,217,21:02:58.1268007,2.471564,1.725951,0,0.000000
1,4,2,218,21:02:58.1608752,2.549733,1.485222,34075,0.004425
2,4,2,219,21:02:58.1937998,2.767144,1.206786,66999,0.008701
3,4,2,220,21:02:58.2267801,3.361297,0.516222,99980,0.012984
4,4,2,221,21:02:58.2608795,4.150430,0.457089,134079,0.017413
5,4,2,222,21:02:58.2938002,5.149151,1.330842,167000,0.021688
6,4,2,223,21:02:58.3267998,5.811839,2.031135,199999,0.025974
7,4,2,224,21:02:58.3608755,6.997533,3.159335,234075,0.030399
8,4,2,225,21:02:58.3938013,8.186203,4.371523,267001,0.034676
9,4,2,226,21:02:58.4267998,8.993676,5.592493,299999,0.038961


5


,pNum,eNum,frameNum,time,arc_right,arc_left,relativeTime,normalized
0,5,2,225,21:05:21.4601582,5.597186,0.768722,0,0.000000
1,5,2,226,21:05:21.4931665,5.605880,0.739591,33008,0.005001
2,5,2,227,21:05:21.5260915,5.756553,0.721798,65933,0.009990
3,5,2,228,21:05:21.5601756,5.769480,0.561145,100017,0.015154
4,5,2,229,21:05:21.5931811,6.008745,0.271228,133023,0.020155
5,5,2,230,21:05:21.6261006,6.398484,0.171535,165942,0.025143
6,5,2,231,21:05:21.6601550,7.013296,0.737615,199997,0.030303
7,5,2,232,21:05:21.6931637,6.904829,1.743033,233005,0.035304
8,5,2,233,21:05:21.7260887,7.841937,2.602181,265930,0.040293
9,5,2,234,21:05:21.7601743,8.361491,3.861291,300016,0.045457


## Exercise 3

In [4]:
import math 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# import data 
df = pd.read_csv('cleanedData_ex3.csv')      #('combined_P1-5_E1.csv')    # ('/data/pepper/combined.csv')

for person in range(1, 6):
    print(str(person))
    # gets data from 1 person
    one_p = df.loc[df['pNum'] == person]
    one_e = one_p  #.loc[one_p['eNum'] == 3]
#     one_e = one_e.loc[one_e['frameNum'] > 200]
#     one_e = one_e.loc[one_e['frameNum'] < 270]

    ###################
    # ROTATE BODY
    ###################
    # create empty DataFrame
    index = np.arange(0)
    frames = pd.DataFrame(index=index)
    frames = frames.fillna(0) # with 0s rather than NaNs

    # get min and max frameNum
    frame_max = int(one_e['frameNum'].max())
    frame_min = int(one_e['frameNum'].min())

    found = False
    
    for i in range(frame_min, frame_max + 1):   
        frame = one_e.loc[one_e['frameNum'] == i]

        try:
            x_ver = float(frame.loc[frame['jointName'] == "HipRight", 'x'])
            z_ver = float(frame.loc[frame['jointName'] == "HipRight", 'z'])
            
#             if float(frame.loc[frame['jointName'] == "HipRight", 'z']) < float(frame.loc[frame['jointName'] == "HipLeft", 'z']):
#                 # rotate clockwise
#                 clock = -1
#             else:
#                 # rotate - clockwise
#                 clock = 1
            
            # schuiven
            frame['x_schuif'] = frame.apply (lambda row: x_translation(x_ver, row),axis=1)
            frame['z_schuif'] = frame.apply (lambda row: z_translation(z_ver, row),axis=1)
            
            #draaien
#             arc = define_arc(frame)
#             frame['x_rotated'] = frame.apply (lambda row: rotate_body_x(row, arc * clock),axis=1)
#             frame['z_rotated'] = frame.apply (lambda row: rotate_body_z(row, arc * clock),axis=1)

            #samenvoegen
            # all frames from one person are rotated
            frames = frames.append(frame)
            found = True
        except:
            found = False
            print('BUG 1')
#      frames.to_csv('test_rotation.csv')
    ###################
    # CALCULATE ARCS
    ###################
    if found == True:
        # create empty DataFrame
        index = np.arange(0)
        columns = ['pNum', 'eNum', 'frameNum', 'time', 'arc_right', 'arc_left']
        arcs = pd.DataFrame(index=index, columns=columns)
        arcs = arcs.fillna(0) # with 0s rather than NaNs

        frame_max_2 = frames['frameNum'].max()
        frame_min_2 = frames['frameNum'].min()

        for i in range(frame_min_2, frame_max_2 + 1): 
            frame = frames.loc[frames['frameNum'] == i]
            right_arc = arc_elbow_above(frame, 'r')
            left_arc = arc_elbow_above(frame, 'l')
            time = frame.loc[frame['jointName'] == 'SpineBase', 'time']
            time = time[time.first_valid_index()]
            row = {'pNum': person, 'eNum': 3, 'frameNum': i, 'time': time, 'arc_right': right_arc[0,0], 'arc_left': left_arc[0,0]}
            arcs = arcs.append(row, ignore_index=True)

        ##################
        # NORMALIZE TIME
        ##################

        arcs['relativeTime'] = arcs.apply (lambda row: relative_time(row, arcs),axis=1)
        arcs['normalized'] = arcs.apply (lambda row: normalized_time(row, arcs), axis=1)
        
        display(arcs)

#         plt.plot(arcs['normalized'], arcs['arc_left'])
#         plt.plot(arcs['normalized'], arcs['arc_right'])
        
#         plt.xlabel('tijd (relatief)')
#         plt.ylabel('graden')
#         plt.ylim((0,110))
#         plt.legend()
#         plt.savefig('/home/13040367/notebooks/Plots/Exercise_3/grafiek_P' + str(person) + '_ex3')    
#         plt.show()
    else:
        print('BUG 2')

1


/opt/jupyterhub/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/jupyterhub/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,pNum,eNum,frameNum,time,arc_right,arc_left,relativeTime,normalized
0,1,3,196,20:53:44.5964860,8.506857,13.270389,0,0.000000
1,1,3,197,20:53:44.6294896,8.664250,13.281156,33003,0.012856
2,1,3,198,20:53:44.6635677,8.608265,13.297350,67081,0.026131
3,1,3,199,20:53:44.6964875,8.379377,13.391493,100001,0.038955
4,1,3,200,20:53:44.7294986,6.941891,14.241180,133012,0.051814
5,1,3,201,20:53:44.7635613,6.386630,16.006798,167075,0.065084
6,1,3,202,20:53:44.7964867,5.802829,19.811319,200000,0.077909
7,1,3,203,20:53:44.8294871,3.575292,21.410263,233001,0.090765
8,1,3,204,20:53:44.8635790,1.442721,26.676445,267093,0.104045
9,1,3,205,20:53:44.8965056,5.905053,29.841362,300019,0.116871


2


,pNum,eNum,frameNum,time,arc_right,arc_left,relativeTime,normalized
0,2,3,288,20:57:04.3955233,8.267635,19.706309,0,0.000000
1,2,3,289,20:57:04.4285225,8.319207,19.705761,32999,0.008182
2,2,3,290,20:57:04.4626041,8.382637,19.810949,67081,0.016633
3,2,3,291,20:57:04.4955295,8.462998,19.850377,100006,0.024797
4,2,3,292,20:57:04.5285213,8.495026,19.903792,132998,0.032978
5,2,3,293,20:57:04.5625942,8.581841,19.890171,167071,0.041426
6,2,3,294,20:57:04.5955121,8.788491,20.484947,199989,0.049589
7,2,3,295,20:57:04.6285279,9.300423,20.671275,233004,0.057775
8,2,3,296,20:57:04.6625997,10.088318,21.146250,267076,0.066223
9,2,3,297,20:57:04.6955334,10.917491,23.015178,300010,0.074389


...

,pNum,eNum,frameNum,time,arc_right,arc_left,relativeTime,normalized
0,3,3,171,20:59:33.9947959,15.336991,0.917731,0,0.000000
1,3,3,172,20:59:34.0278006,13.901501,3.970752,33005,0.009802
2,3,3,173,20:59:34.0618696,15.689806,0.135676,67074,0.019921
3,3,3,174,20:59:34.0947797,14.109413,0.432659,99984,0.029695
4,3,3,175,20:59:34.1277955,14.188514,0.372330,133000,0.039500
5,3,3,176,20:59:34.1618724,15.925939,0.569527,167077,0.049621
6,3,3,177,20:59:34.1947974,16.172324,2.134136,200002,0.059400
7,3,3,178,20:59:34.2277970,16.693998,3.673337,233002,0.069201
8,3,3,179,20:59:34.2618715,17.883433,7.311618,267076,0.079320
9,3,3,180,20:59:34.2947966,19.998918,13.789968,300001,0.089099


4


,pNum,eNum,frameNum,time,arc_right,arc_left,relativeTime,normalized
0,4,3,280,21:03:29.4936466,8.256619,11.395658,0,0.000000
1,4,3,281,21:03:29.5266466,8.259932,11.346072,33000,0.006346
2,4,3,282,21:03:29.5607227,8.292732,11.333021,67076,0.012899
3,4,3,283,21:03:29.5936524,8.302451,11.327928,100006,0.019232
4,4,3,284,21:03:29.6266473,8.333311,11.464179,133001,0.025577
5,4,3,285,21:03:29.6607215,9.784668,11.858841,167075,0.032130
6,4,3,286,21:03:29.6936453,8.193874,11.996586,199999,0.038462
7,4,3,287,21:03:29.7266461,8.066011,12.319500,233000,0.044808
8,4,3,288,21:03:29.7607214,10.215422,13.052017,267075,0.051361
9,4,3,289,21:03:29.7936449,8.559077,14.665162,299998,0.057692


5


,pNum,eNum,frameNum,time,arc_right,arc_left,relativeTime,normalized
0,5,3,331,21:05:53.8600006,18.409920,9.563531,0,0.000000
1,5,3,332,21:05:53.8930160,18.542325,9.666434,33016,0.006738
2,5,3,333,21:05:53.9259402,11.708343,9.836836,65940,0.013457
3,5,3,334,21:05:53.9599978,11.401390,9.970210,99997,0.020408
4,5,3,335,21:05:53.9929974,11.155317,10.100614,132997,0.027142
5,5,3,336,21:05:54.0259216,10.996442,10.139938,165921,0.033861
6,5,3,337,21:05:54.0599977,10.850365,10.314926,199997,0.040816
7,5,3,338,21:05:54.0929969,9.268271,10.900871,232996,0.047550
8,5,3,339,21:05:54.1259405,9.079059,11.302399,265940,0.054274
9,5,3,340,21:05:54.1600150,8.726647,12.423118,300015,0.061228
